In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        dropnum = 0.35
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.drop1 = nn.Dropout(dropnum)
        self.drop2 = nn.BatchNorm1d(embedding_size)
        
    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
#         embedded = self.drop2(embedded)
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(iterator):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden
#         print("DECODER INPUT", decoder_input.shape)
#         print("DECODER HIDDEN", decoder_hidden[0].shape)

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 1

for epoch in range(1, num_epoch + 1):
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
39071,"[577.4776000976562, 1377.28271484375]","[577.1995239257812, 1376.1295166015625]","[577.2323608398438, 1375.0042724609375]","[577.2577514648438, 1373.8751220703125]","[577.2811889648438, 1372.731689453125]","[577.298828125, 1371.5616455078125]","[577.3069458007812, 1370.3516845703125]","[577.3011474609375, 1369.08642578125]","[577.2786254882812, 1367.751708984375]","[577.2415771484375, 1366.3397216796875]",...,"[577.791748046875, 1352.0955810546875]","[577.8950805664062, 1351.0784912109375]","[578.0119018554688, 1350.009521484375]","[578.1392211914062, 1348.8812255859375]","[578.267333984375, 1347.7095947265625]","[578.3826293945312, 1346.5301513671875]","[578.4727783203125, 1345.3858642578125]","[578.531982421875, 1344.3133544921875]","[578.5613403320312, 1343.3353271484375]","[578.5669555664062, 1342.460693359375]"
20832,"[2193.87353515625, 741.1361083984375]","[2192.908447265625, 740.7027587890625]","[2192.4365234375, 740.0703125]","[2191.84716796875, 739.3858642578125]","[2191.23095703125, 738.7315063476562]","[2190.59814453125, 738.08740234375]","[2189.96044921875, 737.4569091796875]","[2189.31787109375, 736.8339233398438]","[2188.66796875, 736.211669921875]","[2188.007080078125, 735.5843505859375]",...,"[2180.66650390625, 728.9508056640625]","[2180.0361328125, 728.3533935546875]","[2179.396728515625, 727.71728515625]","[2178.740966796875, 727.0335083007812]","[2178.060546875, 726.308837890625]","[2177.353759765625, 725.5562133789062]","[2176.623779296875, 724.79345703125]","[2175.880126953125, 724.0355834960938]","[2175.137451171875, 723.291259765625]","[2174.412353515625, 722.562744140625]"
281,"[1823.029052734375, 433.646484375]","[1821.5068359375, 432.7584228515625]","[1820.4713134765625, 431.6373596191406]","[1819.4169921875, 430.7631530761719]","[1818.4356689453125, 429.9237060546875]","[1817.4482421875, 429.0968933105469]","[1816.4276123046875, 428.2197265625]","[1815.340576171875, 427.2648620605469]","[1814.182373046875, 426.2303161621094]","[1812.9744873046875, 425.15216064453125]",...,"[1801.594482421875, 414.1127014160156]","[1800.28369140625, 412.981689453125]","[1799.1533203125, 412.14300537109375]","[1798.2337646484375, 411.4197998046875]","[1797.402587890625, 410.809326171875]","[1796.6390380859375, 410.1955261230469]","[1795.910400390625, 409.5059814453125]","[1795.2052001953125, 408.6766662597656]","[1794.5181884765625, 407.67205810546875]","[1793.854248046875, 406.4963684082031]"
14320,"[739.6670532226562, 1376.80224609375]","[739.501220703125, 1377.0577392578125]","[739.6172485351562, 1378.027099609375]","[739.3985595703125, 1378.703125]","[739.4227294921875, 1379.5406494140625]","[739.26318359375, 1380.2574462890625]","[739.1983642578125, 1381.0107421875]","[739.0748291015625, 1381.7177734375]","[738.9851684570312, 1382.421875]","[738.888671875, 1383.1048583984375]",...,"[738.3099365234375, 1390.0823974609375]","[738.2680053710938, 1390.7373046875]","[738.2235107421875, 1391.405517578125]","[738.1755981445312, 1392.0867919921875]","[738.123291015625, 1392.780029296875]","[738.06591796875, 1393.48388671875]","[738.0027465820312, 1394.19677734375]","[737.9332275390625, 1394.917236328125]","[737.8568725585938, 1395.64453125]","[737.7733764648438, 1396.378173828125]"
36564,"[1760.8182373046875, 379.51373291015625]","[1762.6429443359375, 380.94573974609375]","[1763.9720458984375, 382.3204345703125]","[1765.2413330078125, 383.44775390625]","[1766.51123046875, 384.5288391113281]","[1767.8370361328125, 385.61279296875]","[1769.2510986328125, 386.7601623535156]","[1770.7564697265625, 388.0345764160156]","[1772.3287353515625, 389.4031677246094]","[1773.9024658203125, 390.79144287109375]",...,"[1788.010986328125, 402.9591369628906]","[1789.180419921875, 404.1715087890625]","[1790.32568359375, 405.4411315917969]","[1791.3802490234375, 406.64117431640625]","[1792.261474609375, 407.686279296875]","[1792.9158935546875, 408.5484924316406]","[1793.34887695312

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
39071,577.477600,1377.282715,577.199524,1376.129517,577.232361,1375.004272,577.257751,1373.875122,577.281189,1372.731689,...,578.382629,1346.530151,578.472778,1345.385864,578.531982,1344.313354,578.561340,1343.335327,578.566956,1342.460693
20832,2193.873535,741.136108,2192.908447,740.702759,2192.436523,740.070312,2191.847168,739.385864,2191.230957,738.731506,...,2177.353760,725.556213,2176.623779,724.793457,2175.880127,724.035583,2175.137451,723.291260,2174.412354,722.562744
281,1823.029053,433.646484,1821.506836,432.758423,1820.471313,431.637360,1819.416992,430.763153,1818.435669,429.923706,...,1796.639038,410.195526,1795.910400,409.505981,1795.205200,408.676666,1794.518188,407.672058,1793.854248,406.496368
14320,739.667053,1376.802246,739.501221,1377.057739,739.617249,1378.027100,739.398560,1378.703125,739.422729,1379.540649,...,738.065918,1393.483887,738.002747,1394.196777,737.933228,1394.917236,737.856873,1395.644531,737.773376,1396.378174
36564,1760.818237,379.513733,1762.642944,380.945740,1763.972046,382.320435,1765.241333,383.447754,1766.511230,384.528839,...,1792.915894,408.548492,1793.348877,409.233246,1793.609741,409.759766,1793.758179,410.183838,1793.837769,410.523346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29056,1755.286743,375.244995,1756.472534,376.033203,1757.203979,376.810883,1757.985962,377.591125,1758.798950,378.395630,...,1776.817627,394.801941,1777.702637,395.643890,1778.586060,396.464691,1779.469727,397.256653,1780.358887,398.014282
6181,1713.620972,492.050873,1713.551025,492.379028,1713.170166,492.805969,1712.977783,493.269684,1712.750732,493.723816,...,1704.776978,502.520355,1704.431885,503.015808,1704.107056,503.522247,1703.803467,504.039703,1703.521118,504.568237
10200,2162.483154,779.860962,2162.492432,777.887085,2163.336670,776.549377,2164.147217,775.276428,2164.867432,773.995850,...,2167.818115,752.587219,2167.479736,751.866333,2167.159668,751.203003,2166.863525,750.599182,2166.593506,750.053894


In [12]:
df2.to_csv("outputs1epb100.csv", index=True)